In [1]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import warnings
warnings.filterwarnings("ignore")

import os
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt
from scipy.stats.mstats import gmean
%matplotlib inline
from subprocess import check_output
print(check_output(["ls", "data/subs/team_subs"]).decode("utf8"))

# Any results you write to the current directory are saved as output.

bestpublic.csv
lgbm_1945.csv
mpnn_1884.csv
stack_median_lgbm_mpnn_semgcn.csv
stack_median_lgbm_mpnn_semgcn_updated.csv_2080.csv
weighted_average_lgbm_semgcn_mpnn_2001.csv



In [2]:
sub_path = "data/subs/team_subs"
all_files = os.listdir(sub_path)
all_files = ['mpnn_1884.csv', 'lgbm_1945.csv']

In [3]:
import warnings
warnings.filterwarnings("ignore")
outs = [pd.read_csv(os.path.join(sub_path, f), index_col=0) for f in all_files]
concat_sub = pd.concat(outs, axis=1)
cols = list(map(lambda x: "mol" + str(x), range(len(concat_sub.columns))))
concat_sub.columns = cols
concat_sub.reset_index(inplace=True)
ncol = concat_sub.shape[1]
concat_sub.head()

,id,mol0,mol1
0,4658147,12.473142,18.680381
1,4658148,154.214188,199.365320
2,4658149,3.837926,11.139585
3,4658150,154.214188,199.412384
4,4658151,12.473142,18.680381


In [4]:
# check correlation
concat_sub.iloc[:,1:].corr()

,mol0,mol1
mol0,1.000000,0.999917
mol1,0.999917,1.000000


### Best prediction per type

In [5]:
test = pd.read_csv("data/test.csv")
test.head()

,id,molecule_name,atom_index_0,atom_index_1,type
0,4658147,dsgdb9nsd_000004,2,0,2JHC
1,4658148,dsgdb9nsd_000004,2,1,1JHC
2,4658149,dsgdb9nsd_000004,2,3,3JHH
3,4658150,dsgdb9nsd_000004,3,0,1JHC
4,4658151,dsgdb9nsd_000004,3,1,2JHC


In [6]:
sub = test.copy()
sub['scalar_coupling_constant'] = 0

In [7]:
#mol0 == mpnn
#mol1 == lightgbm

In [8]:
concat_sub.head()

,id,mol0,mol1
0,4658147,12.473142,18.680381
1,4658148,154.214188,199.365320
2,4658149,3.837926,11.139585
3,4658150,154.214188,199.412384
4,4658151,12.473142,18.680381


In [9]:
output_file = 'best_model_per_type_lgbm_mpnn.csv'

df_test = pd.read_csv('data/test.csv')

df_test = df_test.sort_values(by=['id'])

test_id = df_test.id.values
test_type = df_test.type.values

num_test = len(test_id)
coupling_count = np.zeros(num_test)
coupling_value = np.zeros(num_test)

In [10]:
subs = [(['1JHC', '1JHN', '2JHN', '2JHC', '3JHC', '3JHN'],
         'data/subs/team_subs/mpnn_1884.csv'),
        (['2JHH', '3JHH'],
         'data/subs/team_subs/lgbm_1945.csv')]

for valid_type, f in subs:
    df = pd.read_csv(f)
    df = df.sort_values(by=['id'])
    for t in valid_type:
        index = np.where(test_type == t)[0]
        coupling_value[index] += df.scalar_coupling_constant.values[index]
        coupling_count[index] += 1
    if len(coupling_count[coupling_count>1]) > 0:
        print('overlap in types!')

coupling_value = coupling_value/coupling_count

df = pd.DataFrame(list(zip(test_id, coupling_value)), columns=['id', 'scalar_coupling_constant'])

### manually looking if submission is correct

In [11]:
test[test.type == '2JHH'].head()

,id,molecule_name,atom_index_0,atom_index_1,type
7,4658154,dsgdb9nsd_000015,3,4,2JHH
8,4658155,dsgdb9nsd_000015,3,5,2JHH
11,4658158,dsgdb9nsd_000015,4,5,2JHH
16,4658163,dsgdb9nsd_000015,6,7,2JHH
17,4658164,dsgdb9nsd_000015,6,8,2JHH


In [12]:
mpnn = pd.read_csv('data/subs/team_subs/mpnn_1884.csv')
mpnn[mpnn.id == 4658154]

,id,scalar_coupling_constant
7,4658154,-7.697277


In [13]:
lgbm = pd.read_csv('data/subs/team_subs/lgbm_1945.csv')
lgbm[lgbm.id == 4658154]

,id,scalar_coupling_constant
7,4658154,-7.484122


In [14]:
df[df.id == 4658154]

,id,scalar_coupling_constant
7,4658154,-7.484122


In [15]:
test[test.type == '3JHH'].head()

,id,molecule_name,atom_index_0,atom_index_1,type
2,4658149,dsgdb9nsd_000004,2,3,3JHH
27,4658174,dsgdb9nsd_000016,3,5,3JHH
28,4658175,dsgdb9nsd_000016,3,6,3JHH
29,4658176,dsgdb9nsd_000016,3,7,3JHH
30,4658177,dsgdb9nsd_000016,3,8,3JHH


In [16]:
mpnn = pd.read_csv('data/subs/team_subs/mpnn_1884.csv')
mpnn[mpnn.id == 4658149]

,id,scalar_coupling_constant
2,4658149,3.837926


In [17]:
lgbm = pd.read_csv('data/subs/team_subs/lgbm_1945.csv')
lgbm[lgbm.id == 4658149]

,id,scalar_coupling_constant
2,4658149,11.139585


In [18]:
df[df.id == 4658149]

,id,scalar_coupling_constant
2,4658149,11.139585


In [20]:
test[test.type == '1JHC'].head()

,id,molecule_name,atom_index_0,atom_index_1,type
1,4658148,dsgdb9nsd_000004,2,1,1JHC
3,4658150,dsgdb9nsd_000004,3,0,1JHC
5,4658152,dsgdb9nsd_000015,3,0,1JHC
9,4658156,dsgdb9nsd_000015,4,0,1JHC
12,4658159,dsgdb9nsd_000015,5,0,1JHC


In [21]:
mpnn = pd.read_csv('data/subs/team_subs/mpnn_1884.csv')
mpnn[mpnn.id == 4658148]

,id,scalar_coupling_constant
1,4658148,154.214188


In [22]:
lgbm = pd.read_csv('data/subs/team_subs/lgbm_1945.csv')
lgbm[lgbm.id == 4658148]

,id,scalar_coupling_constant
1,4658148,199.36532


In [23]:
df[df.id == 4658148]

,id,scalar_coupling_constant
1,4658148,154.214188


In [19]:
df[['id', 'scalar_coupling_constant']].to_csv(output_file, index=False, float_format='%.6f')